In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import uproot

In [ ]:
energy_bins = np.append(np.linspace(0, 2, 21), 1e9) # matches Dark Neutrino e+e-, https://microboone-docdb.fnal.gov/cgi-bin/sso/ShowDocument?docid=41219
cos_bins = np.concatenate([np.linspace(-1, 0, 3), np.linspace(0, 0.8, 7), np.linspace(0.8, 0.9, 10), np.linspace(0.9, 1, 10)])
z_bins = np.linspace(0, 1036.8, 11)


In [ ]:
f = uproot.open("/nevis/riverside/data/leehagaman/ngem/data_files/checkout_isotropic_one_gamma_run45_reco2_prod_reco2_hist_4a.root")
iso_truth_df = f["wcpselection"]["T_PFeval"].arrays([
    "truth_pdg",
    "truth_mother",
    "truth_startMomentum",
    "truth_startXYZT",
    ],library="pd")

iso_truth_df

In [ ]:
truth_pdgs = iso_truth_df["truth_pdg"].to_numpy()
truth_mothers = iso_truth_df["truth_mother"].to_numpy()
truth_start_momentums = iso_truth_df["truth_startMomentum"].to_numpy()
truth_start_xyzts = iso_truth_df["truth_startXYZT"].to_numpy()


In [ ]:
energies = []
costhetas = []
zs = []
for event_i in tqdm(range(len(iso_truth_df))):
    energy = np.nan
    costheta = np.nan
    for j in range(len(truth_pdgs[event_i])):
        if truth_mothers[event_i][j] == 0 and truth_pdgs[event_i][j] == 22:
            four_momentum = truth_start_momentums[event_i][j]
            three_momentum = four_momentum[:3]
            xy_momentum = np.linalg.norm(three_momentum[:2])
            z_momentum = three_momentum[2]
            energy = four_momentum[3]
            costheta = z_momentum / np.linalg.norm(three_momentum)
            z = truth_start_xyzts[event_i][j][2]
    energies.append(energy)
    costhetas.append(costheta)
    zs.append(z)

iso_truth_df["prim_photon_energy"] = energies
iso_truth_df["prim_photon_costheta"] = costhetas
iso_truth_df["prim_photon_z"] = zs

benchmark_1_energies = iso_truth_df["prim_photon_energy"].to_numpy()
benchmark_1_cosines = iso_truth_df["prim_photon_costheta"].to_numpy()
benchmark_1_zs = iso_truth_df["prim_photon_z"].to_numpy()
benchmark_1_weights = [0.001] * len(benchmark_1_energies)


In [ ]:
"""benchmark_1_energies = []
benchmark_1_cosines = []
benchmark_1_zs = []
benchmark_1_weights = []
with open("/nevis/riverside/data/karan/TMM_Karan/hepevt_filtered_mass_0.48_tmm_2.5e-06.txt", "r") as f:
    lines = f.readlines()
    for line in tqdm(lines):
        if len(line.split()) == 15:
            final_state_type, pdg, mother_1, mother_2, daughter_1, daughter_2, momentum_x, momentum_y, momentum_z, energy, mass, position_x, position_y, position_z, time = line.split()
            final_state_type = int(final_state_type)
            pdg = int(pdg)
            mother_1 = int(mother_1)
            mother_2 = int(mother_2)
            daughter_1 = int(daughter_1)
            daughter_2 = int(daughter_2)
            momentum_x = float(momentum_x)
            momentum_y = float(momentum_y)
            momentum_z = float(momentum_z)
            energy = float(energy)
            mass = float(mass)
            position_x = float(position_x)
            position_y = float(position_y)
            position_z = float(position_z)
            if pdg == 22:
                benchmark_1_energies.append(energy)
                benchmark_1_cosines.append(momentum_z / np.sqrt(momentum_x**2 + momentum_y**2 + momentum_z**2))
                benchmark_1_zs.append(position_z)
            if pdg == 0: # fake particle to save weights
                benchmark_1_weights.append(energy)"""

benchmark_2_energies = []
benchmark_2_cosines = []
benchmark_2_zs = []
benchmark_2_weights = []
with open("/nevis/riverside/data/karan/TMM_Karan/hepevt_filtered_mass_0.1_tmm_2.5e-07.txt", "r") as f:
    lines = f.readlines()
    for line in tqdm(lines):
        if len(line.split()) == 15:
            final_state_type, pdg, mother_1, mother_2, daughter_1, daughter_2, momentum_x, momentum_y, momentum_z, energy, mass, position_x, position_y, position_z, time = line.split()
            final_state_type = int(final_state_type)
            pdg = int(pdg)
            mother_1 = int(mother_1)
            mother_2 = int(mother_2)
            daughter_1 = int(daughter_1)
            daughter_2 = int(daughter_2)
            momentum_x = float(momentum_x)
            momentum_y = float(momentum_y)
            momentum_z = float(momentum_z)
            energy = float(energy)
            mass = float(mass)
            position_x = float(position_x)
            position_y = float(position_y)
            position_z = float(position_z)
            if pdg == 22:
                benchmark_2_energies.append(energy)
                benchmark_2_cosines.append(momentum_z / np.sqrt(momentum_x**2 + momentum_y**2 + momentum_z**2))
                benchmark_2_zs.append(position_z)
            if pdg == 0: # fake particle to save weights
                benchmark_2_weights.append(energy)

benchmark_1_df = pd.DataFrame({"energy": benchmark_1_energies, "cosine": benchmark_1_cosines, "z": benchmark_1_zs})
benchmark_2_df = pd.DataFrame({"energy": benchmark_2_energies, "cosine": benchmark_2_cosines, "z": benchmark_2_zs})


In [ ]:
benchmark_1_df

In [ ]:
benchmark_2_df

In [ ]:
benchmark_1_energy_bin_indices = np.digitize(benchmark_1_energies, energy_bins) - 1
benchmark_1_cosine_bin_indices = np.digitize(benchmark_1_cosines, cos_bins) - 1
benchmark_1_z_bin_indices = np.digitize(benchmark_1_zs, z_bins) - 1

print(np.min(benchmark_1_energy_bin_indices), np.max(benchmark_1_energy_bin_indices))
print(np.min(benchmark_1_cosine_bin_indices), np.max(benchmark_1_cosine_bin_indices))
print(np.min(benchmark_1_z_bin_indices), np.max(benchmark_1_z_bin_indices))

num_energy_bins = len(energy_bins) - 1
num_cos_bins = len(cos_bins) - 1
num_z_bins = len(z_bins) - 1

# Clamp out-of-range indices to valid ranges
benchmark_1_energy_bin_indices = np.clip(benchmark_1_energy_bin_indices, 0, num_energy_bins - 1)
benchmark_1_cosine_bin_indices = np.clip(benchmark_1_cosine_bin_indices, 0, num_cos_bins - 1)
benchmark_1_z_bin_indices = np.clip(benchmark_1_z_bin_indices, 0, num_z_bins - 1)

# Build flattened indices after clamping
benchmark_1_flattened_3d_bin_indices = (
    benchmark_1_energy_bin_indices * (num_cos_bins) * (num_z_bins)
    + benchmark_1_cosine_bin_indices * (num_z_bins)
    + benchmark_1_z_bin_indices
).astype(np.int64)

benchmark_1_counts = np.histogramdd(
    (benchmark_1_energies, benchmark_1_cosines, benchmark_1_zs),
    bins=(energy_bins, cos_bins, z_bins),
    weights=benchmark_1_weights
)[0]
benchmark_2_counts = np.histogramdd(
    (benchmark_2_energies, benchmark_2_cosines, benchmark_2_zs),
    bins=(energy_bins, cos_bins, z_bins),
    weights=benchmark_2_weights
)[0]

# Safe division: avoid divide-by-zero warnings and fill 0 where denom is 0
reweighting_ratios = np.divide(
    benchmark_2_counts,
    benchmark_1_counts,
    out=np.zeros_like(benchmark_2_counts, dtype=float),
    where=benchmark_1_counts > 0
)

# Flatten in C-order to match manual indexing scheme
reweighting_ratios_flattened = reweighting_ratios.ravel()

# Ensure weights are numpy array for later elementwise multiplication
benchmark_1_weights = np.asarray(benchmark_1_weights, dtype=float)

# Map each event to its ratio
benchmark_1_to_2_reweighting_ratios = reweighting_ratios_flattened[benchmark_1_flattened_3d_bin_indices]


In [ ]:
collapsed_z_bench1 = np.sum(benchmark_1_counts, axis=2)
collapsed_z_bench2 = np.sum(benchmark_2_counts, axis=2)

# Safe division for collapsed ratios
collapsed_z_reweighting_ratios = np.divide(
    collapsed_z_bench2,
    collapsed_z_bench1,
    out=np.zeros_like(collapsed_z_bench2, dtype=float),
    where=collapsed_z_bench1 > 0
)

from matplotlib.colors import LogNorm

plt.figure(figsize=(10, 6))
# log norm with min=1e-3
mesh = plt.pcolormesh(cos_bins, energy_bins, collapsed_z_bench1, shading='auto', norm=LogNorm(vmin=1e-3, vmax=1e4))
plt.colorbar(mesh)
plt.xlabel("cos bin")
plt.ylabel("energy bin")
plt.title("collapsed z bench1")
plt.ylim(0, energy_bins[-2])
plt.show()

plt.figure(figsize=(10, 6))
mesh = plt.pcolormesh(cos_bins, energy_bins, collapsed_z_bench2, shading='auto', norm=LogNorm(vmin=1e-3, vmax=1e4))
plt.colorbar(mesh)
plt.xlabel("cos bin")
plt.ylabel("energy bin")
plt.title("collapsed z bench2")
plt.ylim(0, energy_bins[-2])
plt.show()

plt.figure(figsize=(10, 6))
mesh = plt.pcolormesh(cos_bins, energy_bins, collapsed_z_reweighting_ratios, shading='auto', vmin=0, vmax=2, cmap="bwr")
plt.colorbar(mesh)
plt.xlabel("cos bin")
plt.ylabel("energy bin")
plt.title("collapsed reweighting ratios")
plt.ylim(0, energy_bins[-2])
plt.show()

plt.figure(figsize=(10, 6))
mesh = plt.pcolormesh(cos_bins, energy_bins, collapsed_z_reweighting_ratios, shading='auto', vmin=0, vmax=2, cmap="bwr")
plt.colorbar(mesh)
plt.xlim(0.9, 1)
plt.xlabel("cos bin")
plt.ylabel("energy bin")
plt.title("collapsed reweighting ratios")
plt.ylim(0, energy_bins[-2])
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(benchmark_1_energies, bins=energy_bins, weights=benchmark_1_weights, histtype="step", label="Isotropic 1g sample")
plt.hist(benchmark_2_energies, bins=energy_bins, weights=benchmark_2_weights, histtype="step", label="TMM 0.1 2.5e-7")
plt.hist(benchmark_1_energies, bins=energy_bins, weights=benchmark_1_weights*benchmark_1_to_2_reweighting_ratios, histtype="step", label="Isotropic 1g sample reweighted to TMM 0.1 2.5e-7")
plt.legend()
plt.xlim(0, energy_bins[-2])
plt.yscale("log")
plt.xlabel("energy")
plt.ylabel("counts")
plt.show()

plt.figure(figsize=(10, 6))
plt.hist(benchmark_1_cosines, bins=cos_bins, weights=benchmark_1_weights, histtype="step", label="Isotropic 1g sample")
plt.hist(benchmark_2_cosines, bins=cos_bins, weights=benchmark_2_weights, histtype="step", label="TMM 0.1 2.5e-7")
plt.hist(benchmark_1_cosines, bins=cos_bins, weights=benchmark_1_weights*benchmark_1_to_2_reweighting_ratios, histtype="step", label="Isotropic 1g sample reweighted to TMM 0.1 2.5e-7")
plt.legend()
plt.yscale("log")
plt.xlabel("cosine")
plt.ylabel("counts")
plt.show()

plt.figure(figsize=(10, 6))
plt.hist(benchmark_1_zs, bins=z_bins, weights=benchmark_1_weights, histtype="step", label="Isotropic 1g sample")
plt.hist(benchmark_2_zs, bins=z_bins, weights=benchmark_2_weights, histtype="step", label="TMM 0.1 2.5e-7")
plt.hist(benchmark_1_zs, bins=z_bins, weights=benchmark_1_weights*benchmark_1_to_2_reweighting_ratios, histtype="step", label="Isotropic 1g sample reweighted to TMM 0.1 2.5e-7")
plt.legend()
plt.xlabel("z")
plt.yscale("log")
plt.ylabel("counts")
plt.show()